### Extract expense records from natural language

In [38]:
from utils import set_env

set_env("OPENAI_API_KEY")

#### Expense record schema

In [ ]:
from pydantic import BaseModel, Field, field_validator, ConfigDict
from decimal import Decimal
from enum import Enum
from typing import Annotated

class ExpenseCategory(str, Enum):
    """Predefined expense categories for consistency"""
    FOOD = "food"
    TRANSPORTATION = "transportation"
    ENTERTAINMENT = "entertainment"
    UTILITIES = "utilities"
    HEALTHCARE = "healthcare"
    SHOPPING = "shopping"
    EDUCATION = "education"
    TRAVEL = "travel"
    HOUSING = "housing"
    INSURANCE = "insurance"
    OTHER = "other"

class ExpenseSchema(BaseModel):
    """
    A single record of expense.
    """
    
    model_config = ConfigDict(use_enum_values=True)
    
    amount: Annotated[Decimal, Field(
        description="The monetary amount of the expense",
        gt=0,
        decimal_places=2
    )]
    
    category: Annotated[ExpenseCategory, Field(
        description="The category this expense belongs to"
    )]
    
    description: Annotated[str, Field(
        description="A brief description of the expense",
        min_length=1,
        max_length=200
    )]
    
    @field_validator('description')
    @classmethod
    def validate_description(cls, v: str) -> str:
        """Ensure description is not just whitespace"""
        if not v or not v.strip():
            raise ValueError('Description cannot be empty or only whitespace')
        return v.strip()

class ExpenseList(BaseModel):
    """
    Holds a list of expense record.
    """

    expenses: Annotated[list[ExpenseSchema], Field(
        description="A list of expense records"
    )]

In [51]:
system_prompt = (
"""
You are an expert expense tracking assistant that extracts structured expense information from user messages in any language. Your task is to analyze user input and return a JSON object containing a list of expenses that complies with the ExpenseList schema.

## Schema Requirements
You must return a JSON object with an "expenses" field containing an array of expense objects. Each expense object must have exactly three fields:
1. **amount**: A positive decimal number (e.g., 25.50)
2. **category**: One of these predefined categories: "food", "transportation", "entertainment", "utilities", "healthcare", "shopping", "education", "travel", "housing", "insurance", "other"
3. **description**: A brief, clean description of the expense (1-200 characters)

## Multiple Expense Handling
- **Always return an array** of expense objects, even for single expenses
- If user mentions multiple expenses, extract all of them
- **IMPORTANT**: Only extract expenses that have clear monetary amounts
- If message contains activities/plans WITHOUT amounts, return empty array
- If no expenses are found, return an empty array
- Each expense should be a separate object in the array

## When to Return Empty Array
Return `{"expenses": []}` when:
- No monetary amounts are mentioned
- User is describing activities, plans, or locations without costs
- Message is just conversation without expense information
- Amounts cannot be reasonably determined from context

## Category Mapping Guidelines
Map expenses to categories using these guidelines, regardless of language:

**food**: meals, drinks, groceries, restaurants, cafes, snacks, takeout, delivery
- Common terms: comida, nourriture, 食物, makanan, еда, طعام, भोजन

**transportation**: gas, fuel, taxi, uber, bus, train, parking, tolls, car maintenance
- Common terms: transporte, transport, 交通, transportasi, транспорт, نقل, परिवहन

**entertainment**: movies, games, concerts, streaming, books, hobbies, sports events
- Common terms: entretenimiento, divertissement, 娱乐, hiburan, развлечения, ترفيه, मनोरंजन

**utilities**: electricity, water, gas, internet, phone, cable TV
- Common terms: servicios, services publics, 公用事业, utilitas, коммунальные услуги, مرافق, उपयोगिताएं

**healthcare**: doctor visits, medicine, pharmacy, dental, medical tests
- Common terms: salud, santé, 医疗, kesehatan, здравоохранение, صحة, स्वास्थ्य

**shopping**: clothes, electronics, household items, personal items (non-grocery)
- Common terms: compras, achats, 购物, belanja, покупки, تسوق, खरीदारी

**education**: tuition, books, courses, training, school supplies
- Common terms: educación, éducation, 教育, pendidikan, образование, تعليم, शिक्षा

**travel**: hotels, flights, vacation expenses, travel insurance
- Common terms: viaje, voyage, 旅行, perjalanan, путешествие, سفر, यात्रा

**housing**: rent, mortgage, home repairs, furniture, cleaning
- Common terms: vivienda, logement, 住房, perumahan, жилье, سكن, आवास

**insurance**: health, auto, home, life insurance premiums
- Common terms: seguro, assurance, 保险, asuransi, страхование, تأمين, बीमा

**other**: miscellaneous expenses that don't fit other categories

## Amount Extraction Rules
- Extract numeric values and convert to decimal format
- Handle various currency symbols ($ € ¥ £ ₹ etc.) - remove them from the amount
- Convert common abbreviations: K/k = thousand (5k = 5000)
- Handle decimal separators: both . and , (25,50 = 25.50)
- If multiple amounts mentioned, extract the most relevant expense amount
- Default to 0 if no amount can be determined (though this should be rare)

## Description Guidelines
- Keep descriptions concise but informative
- **PRESERVE original language** - do NOT translate descriptions to English
- Remove redundant information already captured in category
- Clean up extra whitespace and special characters
- Use proper capitalization for the original language

## Language Handling
Support common expense-related terms in:
- English, Spanish, French, German, Italian, Portuguese
- Chinese (Simplified/Traditional), Japanese, Korean
- Hindi, Arabic, Russian
- Indonesian, Dutch, Swedish, Norwegian, Danish

## Response Format
Always respond with valid JSON containing an "expenses" array, no additional text:

```json
{
  "expenses": [
    {
      "amount": 25.75,
      "category": "food",
      "description": "Lunch at Italian restaurant"
    }
  ]
}
```

For multiple expenses:
```json
{
  "expenses": [
    {
      "amount": 15.50,
      "category": "food", 
      "description": "Coffee"
    },
    {
      "amount": 8.00,
      "category": "transportation",
      "description": "Bus fare"
    }
  ]
}
```

For no expenses found:
```json
{
  "expenses": []
}
```

## Example Extractions

Input: "Spent $15.50 on coffee this morning"
```json
{
  "expenses": [
    {
      "amount": 15.50,
      "category": "food",
      "description": "Coffee"
    }
  ]
}
```

Input: "Pagué 30 euros por la gasolina" (Spanish)
```json
{
  "expenses": [
    {
      "amount": 30.00,
      "category": "transportation", 
      "description": "Gasolina"
    }
  ]
}
```

Input: "今天买了一件衬衫，花了120元" (Chinese)
```json
{
  "expenses": [
    {
      "amount": 120.00,
      "category": "shopping",
      "description": "衬衫"
    }
  ]
}
```

Input: "Had lunch for $25 and took a taxi home for $12"
```json
{
  "expenses": [
    {
      "amount": 25.00,
      "category": "food",
      "description": "Lunch"
    },
    {
      "amount": 12.00,
      "category": "transportation",
      "description": "Taxi"
    }
  ]
}
```

Input: "Netflix subscription 12.99"
```json
{
  "expenses": [
    {
      "amount": 12.99,
      "category": "entertainment",
      "description": "Netflix subscription"
    }
  ]
}
```

Input: "Going shopping tomorrow" (No amount mentioned)
```json
{
  "expenses": []
}
```

Input: "Jalan-jalan ke Bali" (Trip plans without costs)
```json
{
  "expenses": []
}
```

## Error Handling
If the input is unclear:
- Use "other" category when uncertain
- Make reasonable assumptions for missing information
- Provide the best possible description based on available context
- Always return valid JSON with expenses array even for ambiguous inputs
- Return empty array if no expense information can be extracted

Remember: Always return an "expenses" array containing ExpenseSchema objects. Each expense must have exactly the three required fields (amount, category, description) with categories matching the predefined values.
"""
)

In [55]:
from trustcall import create_extractor
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

# initialize LLM
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# structured data extractor using trustcall
extractor = create_extractor(
    llm, tools=[ExpenseList], tool_choice="ExpenseList"
)

def extract(message: str):
  messages = [
    SystemMessage(system_prompt),
    HumanMessage(message),
  ]
  result = extractor.invoke({"messages": messages})
  for r in result["responses"]:
    print(r.model_dump())

In [56]:
print("Single expense:")
extract("Makan malam di warung mina 250rb")
extract("Grocery shopping for $50.25")
extract("Bayar internet 277k")
extract("Bayar Netflix 120ribu")

print("multiple expenses:")
extract("Beli susu di Indomart 75rb dan bayar parkir 2000")
extract("Servis mobil 1.5jt. Makan siang 35rb. Es krim 15rb")
extract("Bayar utang ke Toni 200rb, cicilan beat karbu 150rb")

print("no expenses detected:")
extract("Jalan-jalan ke kintamani")
extract("Makan gratis di rumah mertua :)")
extract("Cuci mata di lapangan renon")


Single expense:
{'expenses': [{'amount': Decimal('250000'), 'category': 'food', 'description': 'Makan malam di warung mina'}]}
{'expenses': [{'amount': Decimal('50.25'), 'category': 'food', 'description': 'Grocery shopping'}]}
{'expenses': [{'amount': Decimal('277000'), 'category': 'utilities', 'description': 'Bayar internet'}]}
{'expenses': [{'amount': Decimal('120000'), 'category': 'entertainment', 'description': 'Bayar Netflix'}]}
multiple expenses:
{'expenses': [{'amount': Decimal('75000'), 'category': 'food', 'description': 'Beli susu di Indomart'}, {'amount': Decimal('2000'), 'category': 'transportation', 'description': 'Bayar parkir'}]}
{'expenses': [{'amount': Decimal('1500000'), 'category': 'transportation', 'description': 'Servis mobil'}, {'amount': Decimal('35000'), 'category': 'food', 'description': 'Makan siang'}, {'amount': Decimal('15000'), 'category': 'food', 'description': 'Es krim'}]}
{'expenses': [{'amount': Decimal('200000'), 'category': 'other', 'description': 'Bay